# Incompressible Model

This script demonstrates how to compute the effective Hamiltonian of AB-type copolymers for the incompressible model

### 1. Import Python modules and set environment variables

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import polymerfts

# OpenMP environment variables
os.environ["MKL_NUM_THREADS"] = "1"  # always 1
os.environ["OMP_STACKSIZE"] = "1G"
os.environ["OMP_MAX_ACTIVE_LEVELS"] = "1"  # 0, 1
os.environ["OMP_NUM_THREADS"] = "2"  # 1 ~ 4

# GPU environment variables
os.environ["LFTS_GPU_NUM_BLOCKS"]  = "256"
os.environ["LFTS_GPU_NUM_THREADS"] = "256"
os.environ["LFTS_NUM_GPUS"] = "1" # 1 ~ 2

### 2. The effective Hamiltonian

To see the derivation, read the following papers.
<!-- [A multi-species exchange model for fully fluctuating polymer field theory simulations](https://aip.scitation.org/doi/pdf/10.1063/1.4900574) -->
[Polymer Field Theory for Multimonomer Incompressible Models: Symmetric Formulation and ABC Systems](https://doi.org/10.1021/acs.macromol.4c02636)

* There are $M$ distinct monomer types.
* ${\alpha_p}$ is the total chain length type $p$ chains.
* Total volume is $V=\frac{nN}{\rho_0}$, and total volume fraction occupied by type $p$ is $V_p=\frac{n_pN_p}{\rho_0}$.
* The volume fraction occupied by type $p$ is $\bar{\phi}_p=\frac{V_p}{V}$. 
* $\bar{N}$ is The invariant polymerization index, and $\sqrt{\bar{N}} = C = \frac{\rho_0}{N}$.
* The effective Hamiltonian $H$ is written as 
\begin{align}
\beta H &= -\sum_p n_p\log{Q_p} +
\rho_0\int d{\bf r}\left[
\sum_{i=1}^{M-1} \frac{M\Omega_i^2}{2\lambda_i} +
\sum_{i=1}^{M-1} \frac{\Omega_i S_i}{\lambda_i}
-\Omega_M + \frac{V}{2M}\left({\bf s}^T{\bf e}-\sum_{i=1}^{M-1} \frac{S_i^2}{\lambda_i}\right) \right],  \ \ \ \ (\textrm{per monomer unit})
\end{align}

* The effective Hamiltonian $H$ can be rewritten in 'per chain' unit,
\begin{align}
N\Omega_i &\rightarrow \Omega_i, \ \ N\lambda_i \rightarrow \lambda_i, \ \ N{\bf s} \rightarrow {\bf s}, \ \ N S_i \rightarrow S_i, \\
\frac{\beta H}{CV} &= -\sum_p \frac{\bar{\phi}_p}{\alpha_p}\log{Q_p} +
\frac{1}{V}\int d{\bf r}\left[
\sum_{i=1}^{M-1} \frac{M\Omega_i^2}{2\lambda_i} +
\sum_{i=1}^{M-1} \frac{\Omega_i S_i}{\lambda_i} 
-\Omega_M + \frac{V}{2M}\left({\bf s}^T{\bf e}-\sum_{i=1}^{M-1} \frac{S_i^2}{\lambda_i}\right) \right].  \ \ \ \ (\textrm{per chain unit})
\end{align}

* The relations between the auxiliary fields $\{\Omega_i\}$ and monomer chemical potential fields $\{W_i\}$ are
\begin{align}
\left[\begin{array}{cc} 
W_1({\bf r})\\
W_2({\bf r})\\
\cdots      \\
W_M({\bf r})\\
\end{array}\right]&=
O
\left[\begin{array}{cc} 
\Omega_1({\bf r}) \\ 
\Omega_2({\bf r}) \\ 
\cdots            \\
\Omega_M({\bf r}) \\ 
\end{array}\right],
\end{align}
where $O$ is the orthogonal matrix of $PXP$, and $OO^T=M$.

### 3. Compute coefficients for Computing Hamiltonian and Functional Derivatives

```class polymerfts.SymmetricPolymerTheory``` computes the coefficents $a_i$ and $b_i$, and $U_{ref}$ for given parameters in 'per chain' unit.
\begin{align}
\frac{\beta H}{CV} &= -\sum_p \frac{\bar{\phi}_p}{\alpha_p}\log{Q_p} +
\frac{1}{V}\int d{\bf r}\left[
\sum_{i=1}^{M} a_i \Omega_i^2 + \sum_{i=1}^{M} b_i \Omega_i + U_{ref} \right].  \ \ \ \ (\textrm{per chain unit})
\end{align}

The functional derivative w.r.t $\Omega_i({\bf r})$ is
\begin{align}
\frac{1}{CV}\frac{\delta \beta H}{\delta \Omega_i({\bf r})} &= \Phi_i({\bf r}) +
2 a_i \Omega_i({\bf r}) + b_i.  \ \ \ \ (\textrm{per chain unit})
\end{align}
where $\Phi_i({\bf r})= \sum_j O_{ji}\phi_j({\bf r})$

#### Example 1) AB-type polymeric systems

* The effective Hamiltonian $H$ is written as 
\begin{align}
\beta H &= -\sum_p n_p\log{Q_p} +
\rho_0\int d{\bf r}\left[\frac{\Omega_-^2({\bf r})}{\chi} -\Omega_+({\bf r}) + \frac{\chi}{4}\right],  \ \ \ \ (\textrm{per monomer unit}) \\
\frac{\beta H}{CV} &= -\sum_p \frac{\bar{\phi}_p}{\alpha_p}\log{Q_p} +
\frac{1}{V}\int d{\bf r}\left[\frac{\Omega_-^2({\bf r})}{\chi N} -\Omega_+({\bf r}) + \frac{\chi N}{4}\right].  \ \ \ \ (\textrm{per chain unit})
\end{align}

* The functional derivative $\frac{\delta H}{\delta \Omega_-({\bf r})}$ is 
\begin{align}
\frac{\beta}{\rho_0}\frac{\delta H}{\delta \Omega_-({\bf r})} &= \Phi_-({\bf r}) + \frac{2\Omega_-({\bf r})}{\chi},  \ \ \ \ (\textrm{per monomer unit}) \\
\frac{\beta}{C}\frac{\delta H}{\delta \Omega_-({\bf r})} &= \Phi_-({\bf r}) + \frac{2\Omega_-({\bf r})}{\chi N}.  \ \ \ \ (\textrm{per chain unit})
\end{align}

* The functional derivative $\frac{\delta H}{\delta \Omega_+({\bf r})}$ is 
\begin{align}
\frac{\beta}{\rho_0}\frac{\delta H}{\delta \Omega_+({\bf r})} &= \Phi_+({\bf r}) - 1,  \ \ \ \ (\textrm{per monomer unit}) \\
\frac{\beta}{C}\frac{\delta H}{\delta \Omega_+({\bf r})} &= \Phi_+({\bf r}) - 1.  \ \ \ \ (\textrm{per chain unit})
\end{align}

* The relations between the auxiliary fields and monomer chemical potential fields are
\begin{align}
\left[\begin{array}{cc} 
W_A({\bf r})\\
W_B({\bf r})
\end{array}\right]&=
O
\left[\begin{array}{cc} 
\Omega_-({\bf r}) \\ 
\Omega_+({\bf r}) 
\end{array}\right], \\
\left[\begin{array}{cc} 
\phi_A({\bf r})\\
\phi_B({\bf r})
\end{array}\right]&=
O
\left[\begin{array}{cc} 
\Phi_-({\bf r}) \\ 
\Phi_+({\bf r}) 
\end{array}\right],
\end{align}
where $O=\left[\begin{array}{cc} 
1 & 1 \\ 
-1 & 1
\end{array}\right]$.

In [2]:
monomer_types = ["A","B"]
chi_n = {"A,B":20}

# Polymer field theory for Multimonomer system
mpt = polymerfts.SymmetricPolymerTheory(monomer_types, chi_n, zeta_n=None)

One of eigenvalues is zero for given chiN values.
------------ Polymer Field Theory for Multimonomer ------------
Eigenvalues:
	 [-20.   0.]
Eigenvectors [v1, v2, ...] :
	 [[ 1.  1.]
	 [-1.  1.]]
Mapping matrix A:
	 [[ 1.  1.]
	 [-1.  1.]]
Real Fields:  [0]
Imaginary Fields:  [1]
In Hamiltonian:
	reference energy:  5.0
	coefficients of int of mu(r)/V:  [-0. -1.]
	coefficients of int of mu(r)^2/V:  [0.05 0.  ]
	dH_ref/dχN:  {'A,B': 0.24999999999053554}
	d(coef of mu(r))/dχN:  {'A,B': array([0., 0.])}
	d(coef of mu(r)^2)/dχN:  {'A,B': array([-0.0025,  0.    ])}
